# Local copilot

En este post vamos a ver cómo instalar un copilot en local para ahorrarnos el precio de GitHub Copilot y para asegurarnos que nustros datos no se comparten con nadie. Para ello vamos a utilizar [Ollama](https://maximofn.com/ollama/) como proveedor de LLMs y [Continue](https://www.continue.dev/) como plugin para nuestro IDE.

## Modelo de lenguaje

Para elegir el modelo de lenguaje que vamos a usar miramos la tabla de clasificación de modelos de lenguaje para código [evalplus](https://evalplus.github.io/leaderboard.html).

A día de hoy, cuando estoy escribiendo el post, los tres primeros modelos son modelos GPT4, pero el tercero es el modelo `CodeQwen1.5-7B-Chat`, que es un modelo de 7B de parámetros. Por lo que para usar a nivel local, incluso con ordenadores sin GPU, es factible.

Pero el problema de `CodeQwen1.5-7B-Chat` es que su licencia no es libre totalmente para uso comercial, por lo que si quieres usarlo para un uso comercial, o si tienes dudas de si puedes usarlo o no, vamos a ver también los modelos `OpenCodeInterpreter-DS-33B` y `OpenCodeInterpreter-DS-6.7B`. Si tenemos un ordenador con GPUs con bastante memoria podremos usar el de 33B de parámetros, pero si tenemos un ordenador más limitado entonces usaremos el de 6.7B de parámetros.

![EvalPlus_laderboard](https://pub-fb664c455eca46a2ba762a065ac900f7.r2.dev/EvalPlus_laderboard.webp)

Usemos el LLM que usemos, como gestor o proveedor de LLMs vamos a usar [Ollama](https://maximofn.com/ollama/) que ya expliqué en otro [post](https://maximofn.com/ollama/), así que vamos aprovechar el conocimiento para usar modelos cuantizados para que sean más pequeños, se ejecuten más rápido y ocupen menos memoria.

### Modelo `CodeQwen1.5-7B-Chat`

Este modelo es de la empresa `Qwen`, está disponible en la [librería de ollama](https://ollama.com/library/codeqwen), pero además podemos encontrarlo cuantizado en su página de [HuggingFace](https://huggingface.co/Qwen/CodeQwen1.5-7B-Chat-GGUF). Lo podemos descargar cuantizado desde 2 bits hasta 8 bits, así que vamos a quedarnos con un tamaño intermedio de 4 bits. Así que para ello, primero vamos a crear una carpeta donde vamos a descargar el modelo

In [1]:
import pathlib

path = 'ollama_modelfile'
pathlib.Path(path).mkdir(parents=True, exist_ok=True)   # Make folder if it doesn't exist

Ahora nos descargamos el modelo cuantizado. Como hemos dicho vamos a descargar el modelo de 4 bits, pero si prefieres descargar el modelo cuantizado en otro número de bits, puedes ver todos los modelos que hay en la página de [archivos](https://huggingface.co/Qwen/CodeQwen1.5-7B-Chat-GGUF/tree/main) del modelo 

In [2]:
!wget https://huggingface.co/Qwen/CodeQwen1.5-7B-Chat-GGUF/resolve/main/codeqwen-1_5-7b-chat-q4_k_m.gguf -O ollama_modelfile/codeqwen-1_5-7b-chat-q4_k_m.gguf

--2024-06-05 18:48:49--  https://huggingface.co/Qwen/CodeQwen1.5-7B-Chat-GGUF/resolve/main/codeqwen-1_5-7b-chat-q4_k_m.gguf
Resolviendo huggingface.co (huggingface.co)... 54.192.95.21, 54.192.95.79, 54.192.95.70, ...
Conectando con huggingface.co (huggingface.co)[54.192.95.21]:443... conectado.
Petición HTTP enviada, esperando respuesta... 302 Found
Ubicación: https://cdn-lfs-us-1.huggingface.co/repos/6a/7d/6a7d8f0e227de8c36c5c48a9fed818af501675e59f4cec619ac30c7a17303e7e/98572d2cbc355c6be6c89b431df5c26c5bc2838dc755b53fe5f81eb9fa19df3c?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27codeqwen-1_5-7b-chat-q4_k_m.gguf%3B+filename%3D%22codeqwen-1_5-7b-chat-q4_k_m.gguf%22%3B&Expires=1717865329&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNzg2NTMyOX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzZhLzdkLzZhN2Q4ZjBlMjI3ZGU4YzM2YzVjNDhhOWZlZDgxOGFmNTAxNjc1ZTU5ZjRjZWM2MTlhYzMwYzdhMTczMDNlN2UvOTg1NzJkMmNiYz

Ahora, para poder usar este modelo en Ollama, tenemos que crear un `Modelfile`. Ya explicamos en el post de [Ollama](https://maximofn.com/ollama/) cómo crear un `Modelfile`, así que si quires entender lo que vamos a hacer ahora, te recomiendo leerlo, pero si no quieres leerlo, simplemente sigue lo que vamos a hacer.

Lo primero que vamos a hacer es descar, el modelo de la [librería de ollama](https://ollama.com/library/codeqwen) para ver su `Modelfile` y hacerlo parecido

In [3]:
!ollama pull codeqwen

pulling manifest 
pulling 8e31a8360156... 100% ▕██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏ 4.2 GB                         
pulling 6b53223f338a... 100% ▕██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏ 6.9 KB                         
pulling 62fbfd9ed093... 100% ▕██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏  182 B                         
pulling f02dd72bb242... 100% ▕██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏   59 B                         
pulling f662c9193901... 100% ▕██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏  483 B                         
verifying sha256 digest 
writing manifest 
removing 

Ahora leemos su `Modelfile`

In [4]:
!ollama show codeqwen --modelfile

# Modelfile generated by "ollama show"
# To build a new Modelfile based on this, replace FROM with:
# FROM codeqwen:latest

FROM /usr/share/ollama/.ollama/models/blobs/sha256-8e31a8360156f558b83085fdae0952112082000df74426197ab91b88635613b6
TEMPLATE "{{ if .System }}<|im_start|>system
{{ .System }}<|im_end|>
{{ end }}{{ if .Prompt }}<|im_start|>user
{{ .Prompt }}<|im_end|>
{{ end }}<|im_start|>assistant
{{ .Response }}<|im_end|>
"
PARAMETER stop <|im_start|>
PARAMETER stop <|im_end|>
LICENSE """Tongyi Qianwen LICENSE AGREEMENT

Tongyi Qianwen Release Date: August 3, 2023

By clicking to agree or by using or distributing any portion or element of the Tongyi Qianwen Materials, you will be deemed to have recognized and accepted the content of this Agreement, which is effective immediately.

1. Definitions
    a. This Tongyi Qianwen LICENSE AGREEMENT (this "Agreement") shall mean the terms and conditions for use, reproduction, distribution and modification of the Materials as defined by thi

Lo copiamos, y lo pegamos en un `Modelfile` para el modelo cuantizado.

Después vamos a hacer unos cambios:
 * Cambiamos el `FROM` por el modelo que nos hemos descargado
 * Cambiamos la `LICENSE` por una referencia a la página del archivo de licencia de HuggingFace
 * Añadimos un mensaje de sistema

In [5]:
%%writefile ollama_modelfile/Modelfile_codeqwen
FROM ./codeqwen-1_5-7b-chat-q4_k_m.gguf
TEMPLATE "{{ if .System }}<|im_start|>system
{{ .System }}<|im_end|>
{{ end }}{{ if .Prompt }}<|im_start|>user
{{ .Prompt }}<|im_end|>
{{ end }}<|im_start|>assistant
{{ .Response }}<|im_end|>
"
PARAMETER stop <|im_start|>
PARAMETER stop <|im_end|>
SYSTEM """You are a code wizard. Always answer in the language in which you are asked"""
LICENSE """Codeqwen license
https://huggingface.co/Qwen/CodeQwen1.5-7B-Chat/blob/main/LICENSE"""

Writing ollama_modelfile/Modelfile_codeqwen


Por último creamos el modelo desde el `Modelfile` que hemos creado, con el nombre `CodeQwen1.5-7B-Chat-Q4-KM`

In [6]:
!ollama create CodeQwen1.5-7B-Chat-Q4-KM -f ollama_modelfile/Modelfile_codeqwen

transferring model data 
using existing layer sha256:98572d2cbc355c6be6c89b431df5c26c5bc2838dc755b53fe5f81eb9fa19df3c 
using existing layer sha256:62fbfd9ed093d6e5ac83190c86eec5369317919f4b149598d2dbb38900e9faef 
creating new layer sha256:f4c662cc04ca185d87fe783239f5ee26f96c0c74bf71cc566cd01ed9c3d14671 
creating new layer sha256:573587370b225d7a7b0435d87baeca6a4444ed30025c4dc820564ecb492c535e 
using existing layer sha256:f02dd72bb2423204352eabc5637b44d79d17f109fdb510a7c51455892aa2d216 
creating new layer sha256:f3f2858cf8f52a5c145beb41369866d87ab81115e34d8be578c4bf1c3bf727eb 
writing manifest 
success

Ya tenemos el modelo creado, lo podemos comprobar haciendo `ollama list`

In [8]:
!ollama list

NAME                            	ID          	SIZE  	MODIFIED           
CodeQwen1.5-7B-Chat-Q4-KM:latest	4fe061b1854c	4.7 GB	About a minute ago	
codeqwen:latest                 	a6f7662764bd	4.2 GB	14 minutes ago    	
assistanmal:latest              	2be327b28fbf	4.9 GB	12 days ago       	
llama3:70b                      	786f3184aec0	39 GB 	2 weeks ago       	
llava:latest                    	8dd30f6b0cb1	4.7 GB	2 weeks ago       	
moondream:latest                	55fc3abd3867	1.7 GB	2 weeks ago       	
phi3:latest                     	a2c89ceaed85	2.3 GB	2 weeks ago       	
llama3:latest                   	365c0bd3c000	4.7 GB	2 weeks ago       	


Si no vamos a usar el modelo `codeqwen:latest` lo podemos eliminar para que no ocupe memoria

In [9]:
!ollama rm codeqwen:latest

deleted 'codeqwen:latest'


### Modelo `OpenCodeInterpreter-DS-XB`

Este modelo es de la comunidad de investigación de código abierto `M-A-P`.

Este modelo no está disponible en la [librería de ollama](https://ollama.com/library), por lo que directamente vamos a usar uno cuantizado de [Hugging Face](https://huggingface.co/LoneStriker)

Como este modelo no está en la librería de ollama, no podemos copiar su `Modelfile`, pero como es un modelo basado en `Llama3`, vamos a reutilizar el `Modelfile` que hemos usado antes para el modelo codeqwen, que también es un modelo basado en `Llama3`.

Primero descargamos el modelo cuantizado. Igual que antes, hay varios tamaños, pero aquí vamos a quedarnos con el modelo de 4 bits.

Si quieres descargar otro, puedes buscar en la página de [archivos](https://huggingface.co/LoneStriker/OpenCodeInterpreter-DS-6.7B-GGUF/tree/main) del modelo de 6.7B de parámetros, o en la página de [archivos](https://huggingface.co/LoneStriker/OpenCodeInterpreter-DS-33B-GGUF/tree/main) del modelo de 33B de parámetros.

Para descargarlo hacemos

``` bash
wget https://huggingface.co/LoneStriker/OpenCodeInterpreter-DS-XB-GGUF/resolve/main/OpenCodeInterpreter-DS-XB-Q4_K_M.gguf -O ollama_modelfile/OpenCodeInterpreter-DS-XB-Q4_K_M.gguf
```

Cambia el `XB` por `6.7B`, si quieres descargar el modelo de 6.7B de parámetros, o `33B` si quieres descargar el modelo de 33B de parámetros.

Aquí vamos a descargar el modelo de 6.7B de parámetros.

In [11]:
!wget https://huggingface.co/LoneStriker/OpenCodeInterpreter-DS-6.7B-GGUF/resolve/main/OpenCodeInterpreter-DS-6.7B-Q4_K_M.gguf -O ollama_modelfile/OpenCodeInterpreter-DS-6.7B-Q4_K_M.gguf

--2024-06-05 19:31:34--  https://huggingface.co/LoneStriker/OpenCodeInterpreter-DS-6.7B-GGUF/resolve/main/OpenCodeInterpreter-DS-6.7B-Q4_K_M.gguf
Resolviendo huggingface.co (huggingface.co)... 54.192.95.21, 54.192.95.70, 54.192.95.79, ...
Conectando con huggingface.co (huggingface.co)[54.192.95.21]:443... conectado.
Petición HTTP enviada, esperando respuesta... 302 Found
Ubicación: https://cdn-lfs-us-1.huggingface.co/repos/74/63/7463865bd55e4e51499482f3c076e3ff9f10373e64db8fb998f51ee734f9035b/70c9a18a4b21dcc5c468a0a1bc8386005474793e2ff067b98545740162af4c96?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27OpenCodeInterpreter-DS-6.7B-Q4_K_M.gguf%3B+filename%3D%22OpenCodeInterpreter-DS-6.7B-Q4_K_M.gguf%22%3B&Expires=1717867894&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNzg2Nzg5NH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzc0LzYzLzc0NjM4NjViZDU1ZTRlNTE0OTk0ODJmM2MwNzZlM2ZmOWYxMDM3M2U2NGRiOGZiOT

Ahora escribimos el `Modelfile`. Acuérdate de cambiar `33B` por `6.7B` si has descargado el modelo de 33B de parámetros.

In [12]:
%%writefile ollama_modelfile/Modelfile_open_code_interpreter
FROM ./OpenCodeInterpreter-DS-6.7B-Q4_K_M.gguf
TEMPLATE "{{ if .System }}<|im_start|>system
{{ .System }}<|im_end|>
{{ end }}{{ if .Prompt }}<|im_start|>user
{{ .Prompt }}<|im_end|>
{{ end }}<|im_start|>assistant
{{ .Response }}<|im_end|>
"
PARAMETER stop <|im_start|>
PARAMETER stop <|im_end|>
SYSTEM """You are a code wizard. Always answer in the language in which you are asked"""
LICENSE """Apache 2.0 License"""

Writing ollama_modelfile/Modelfile_open_code_interpreter


Ya podemos crear el modelo. Recuerda cambiar `33B` por `6.7B` si has descargado el modelo de 33B de parámetros.

In [13]:
!ollama create OpenCodeInterpreter-DS-6.7B-Q4-KM -f ollama_modelfile/Modelfile_open_code_interpreter

transferring model data 
using existing layer sha256:70c9a18a4b21dcc5c468a0a1bc8386005474793e2ff067b98545740162af4c96 
using existing layer sha256:62fbfd9ed093d6e5ac83190c86eec5369317919f4b149598d2dbb38900e9faef 
using existing layer sha256:f4c662cc04ca185d87fe783239f5ee26f96c0c74bf71cc566cd01ed9c3d14671 
creating new layer sha256:075e5377ffe49ea3415840cb784d85a27084d48632eff977f849c2892e766840 
using existing layer sha256:f02dd72bb2423204352eabc5637b44d79d17f109fdb510a7c51455892aa2d216 
creating new layer sha256:9146d807273bfb83eec116e49f130a2774879b5bba57ee1de1465a031f6e5a03 
writing manifest 
success


Comprobamos que hemos creado bien el modelo

In [14]:
!ollama list

NAME                                    	ID          	SIZE  	MODIFIED           
OpenCodeInterpreter-DS-6.7B-Q4-KM:latest	1c5284fcf99b	4.1 GB	About a minute ago	
CodeQwen1.5-7B-Chat-Q4-KM:latest        	4fe061b1854c	4.7 GB	26 minutes ago    	
assistanmal:latest                      	2be327b28fbf	4.9 GB	12 days ago       	
llama3:70b                              	786f3184aec0	39 GB 	2 weeks ago       	
llava:latest                            	8dd30f6b0cb1	4.7 GB	2 weeks ago       	
moondream:latest                        	55fc3abd3867	1.7 GB	2 weeks ago       	
phi3:latest                             	a2c89ceaed85	2.3 GB	2 weeks ago       	
llama3:latest                           	365c0bd3c000	4.7 GB	2 weeks ago       	


## Plugin 

Una vez hemos creado los modelos que vamos a usar, podemos instalar el plugin [continue](https://docs.continue.dev/). Como en cada IDE se hará de una manera, lo mejor es visitar su página [quickstar](https://docs.continue.dev/quickstart) y seguir las instrucciones.

De hecho, como todo lo que vamos a hacer, va a depender del IDE. Vamos a contarlo de una manera genérica

 > Nota: A mi, al instalarlo en VScode no me cargaba, así que para solucionarlo, en la página del plugin seleccioné `Switch to Pre-Release Version`

### Desactivar la telemetría

Lo primero que tenemos que hacer es desactivar la telemetría del plugin. Para ello cambiamos de `false` el parámetro `allowAnonymousTelemetry` del archivo `~/.continue/config.json`

In [6]:
!cat ~/.continue/config.json

{
  "models": [],
  "customCommands": [
    {
      "name": "test",
      "prompt": "{{{ input }}}\n\nWrite a comprehensive set of unit tests for the selected code. It should setup, run tests that check for correctness including important edge cases, and teardown. Ensure that the tests are complete and sophisticated. Give the tests just as chat output, don't edit any file.",
      "description": "Write unit tests for highlighted code"
    }
  ],
  "tabAutocompleteModel": {
    "title": "Starcoder2 3b",
    "provider": "ollama",
    "model": "starcoder2:3b"
  },
  "allowAnonymousTelemetry": false
}


In [5]:
!sed -i's/"allowAnonymousTelemetry": true/"allowAnonymousTelemetry": false/' ~/.continue/config.json

sed: no se puede abrir el archivo allabot/.continue/config.json: No existe el archivo o el directorio


## Configuración del plugin

Vamos al plugin en el IDE y nos preguntará si queremos usar una API KEY o modelos locales, elegimos modelos locales